# Trueskill

## Calculating Win, Draw probabilities

In [1]:
import numpy as np

from trueskill import Rating, BETA, rate_1vs1, quality_1vs1
from trueskill.backends import cdf

In [2]:
# Give A a rating of 25, and B a rating of 30, both with same standard deviation sigma=0.333
A = Rating()
B = Rating()

In [3]:
# https://stackoverflow.com/questions/27372986/from-match-results-to-rating-using-trueskill-in-python
# A plays B and A wins
A, B = rate_1vs1(A, B)
# A plays B and B wins
B, A = rate_1vs1(B, A)
# A plays B, and it is a Draw
A,B = rate_1vs1(A, B, drawn=True)

In [4]:
# https://stackoverflow.com/questions/28031698/with-the-trueskill-algorithm-given-two-players-ratings-how-can-i-calculate-th

def win_probability(player_rating, opponent_rating):
    delta_mu = player_rating.mu - opponent_rating.mu
    denom = np.sqrt(2 * (BETA * BETA) + pow(player_rating.sigma, 2) + pow(opponent_rating.sigma, 2))
    return cdf(delta_mu / denom)

player_rating = Rating(mu=30, sigma=0.333)
opponent_rating = Rating(mu=25, sigma=0.333)
win_prob = win_probability(player_rating, opponent_rating)
win_prob

0.8011765392965086

In [5]:
def draw_probability(player_rating, opponent_rating):
    return quality_1vs1(player_rating, opponent_rating)

player_rating = Rating(mu=25, sigma=0.333)
opponent_rating = Rating(mu=25, sigma=0.333)
draw_prob = draw_probability(player_rating, opponent_rating)
draw_prob

0.9968216144748854

## Sample Implementation

https://github.com/sublee/trueskill/issues/1#issuecomment-10491635

In [6]:
import random
from math import sqrt
from trueskill import TrueSkill, Rating, quality_1vs1, rate_1vs1, BETA
from trueskill.backends import cdf

In [7]:

def win_probability(team_rating, opponent_rating):
    delta_mu = team_rating.mu - opponent_rating.mu
    denom = sqrt(2 * (BETA * BETA) + pow(team_rating.sigma, 2) + pow(opponent_rating.sigma, 2))
    win_prob = cdf(delta_mu / denom)
    return win_prob

NUM_MATCHES = 5

total_predictions = 0
correct_predictions = 0


class Player:
    def __init__(self, name, talent):
        self.name = name
        self.talent = talent
        self.rating = Rating()
        self.wins = 0
        self.losses = 0
        self.draws = 0

    @property
    def record(self):
        return '%d-%d-%d' % (self.wins, self.losses, self.draws)

    @property
    def mu_sigma(self):
        return '%d~%d' % (round(self.rating.mu * 10), round(self.rating.sigma * 10))

    def __repr__(self):
        return 'Player(%s) %s [%s]' % (self.name, self.record, self.mu_sigma)

# Create players
players = [
    Player('Alice', 50),
    Player('Bob', 60),
    Player('Carol', 66),
    Player('Dave', 67),
    Player('Emily', 67),
    Player('Felix', 75)
]


for n in range(NUM_MATCHES):
    away, home = random.sample(players, 2)
    match_quality = quality_1vs1(away.rating, home.rating)
    away_win_prob = win_probability(away.rating, home.rating)

    if away_win_prob < .33:
        prediction = 'LOSS'
    elif away_win_prob > .66:
        prediction = 'WIN'
    else:
        prediction = 'TOSS-UP'

    print('Matchup: %s (%s) at %s (%s)' % (
        away.name, away.record, home.name, home.record,
    ))
    print('Rating Before: %s / %s' % (away.mu_sigma, home.mu_sigma))
    print('Match Quality: %.2f' % (match_quality))
    print('%s odds winning: %.1f%% [predict: %s]' % (
        away.name,
        away_win_prob * 100,
        prediction
    ))

    away_score = random.randint(0, away.talent)
    home_score = random.randint(0, home.talent)

    is_draw = away_score == home_score

    if is_draw:
        away.draws += 1
        home.draws += 1
        away.rating, home.rating = rate_1vs1(away.rating, home.rating, drawn=True)
        print('%s and %s tied at %d' % (away.name, home.name, away_score))
    else:
        winner = (away_score > home_score) and away or home
        loser = (away_score < home_score) and away or home
        winner.rating, loser.rating = rate_1vs1(winner.rating, loser.rating)
        winner.wins += 1
        loser.losses += 1
        print('%s beats %s (%d - %d)' % (winner.name, loser.name,
            max(away_score, home_score), min(away_score, home_score)))

    # Update prediction
    prediction_result = None
    if prediction == 'WIN':
        total_predictions += 1
        if winner == away:
            correct_predictions += 1
            prediction_result = 'SUCCESS'
        else:
            prediction_result = 'FAIL'
    elif prediction == 'LOSS':
        total_predictions += 1
        if loser == away:
            correct_predictions += 1
            prediction_result = 'SUCCESS'
        else:
            prediction_result = 'FAIL'

    print('Prediction: %s --> %s' % (prediction, prediction_result))
    print('Rating After: %s / %s' % (away.mu_sigma, home.mu_sigma))
    print("\n")

for player in players:
    print(player)

print("Predicted: %d of %d (%.1f%%)" % (
    correct_predictions,
    total_predictions,
    float(correct_predictions) /  total_predictions  * 100
))

Matchup: Dave (0-0-0) at Emily (0-0-0)
Rating Before: 250~83 / 250~83
Match Quality: 0.45
Dave odds winning: 50.0% [predict: TOSS-UP]
Emily beats Dave (55 - 22)
Prediction: TOSS-UP --> None
Rating After: 206~72 / 294~72


Matchup: Emily (1-0-0) at Felix (0-0-0)
Rating Before: 294~72 / 250~83
Match Quality: 0.44
Emily odds winning: 63.8% [predict: TOSS-UP]
Emily beats Felix (61 - 41)
Prediction: TOSS-UP --> None
Rating After: 320~65 / 215~72


Matchup: Carol (0-0-0) at Emily (2-0-0)
Rating Before: 250~83 / 320~65
Match Quality: 0.41
Carol odds winning: 28.2% [predict: LOSS]
Carol beats Emily (66 - 53)
Prediction: LOSS --> FAIL
Rating After: 321~67 / 277~57


Matchup: Carol (1-0-0) at Dave (0-1-0)
Rating Before: 321~67 / 206~72
Match Quality: 0.31
Carol odds winning: 84.4% [predict: WIN]
Dave beats Carol (34 - 12)
Prediction: WIN --> FAIL
Rating After: 260~57 / 277~59


Matchup: Bob (0-0-0) at Felix (0-1-0)
Rating Before: 250~83 / 215~72
Match Quality: 0.45
Bob odds winning: 60.9% [predi

### References

https://gist.github.com/klenwell/3a15eca6b83ce575d0ca

https://stackoverflow.com/questions/28031698/with-the-trueskill-algorithm-given-two-players-ratings-how-can-i-calculate-th

https://stackoverflow.com/questions/27372986/from-match-results-to-rating-using-trueskill-in-python

https://en.wikipedia.org/wiki/TrueSkill
